In [32]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from ResNet import Bottleneck, ResNet, ResNet50

In [33]:
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

In [34]:
train = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)

trainloader = torch.utils.data.DataLoader(train, batch_size=128, shuffle=True, num_workers=2)

test = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

testloader = torch.utils.data.DataLoader(test, batch_size=128,shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [35]:
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [37]:
net = ResNet50(10).to('cuda')

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9, weight_decay=0.0001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor = 0.1, patience=5)

In [38]:
EPOCHS = 200
for epoch in range(EPOCHS):
    losses = []
    running_loss = 0
    for i, inp in enumerate(trainloader):
        inputs, labels = inp
        inputs, labels = inputs.to('cuda'), labels.to('cuda')
        optimizer.zero_grad()
    
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        losses.append(loss.item())

        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if i%100 == 0 and i > 0:
            print(f'Loss [{epoch+1}, {i}](epoch, minibatch): ', running_loss / 100)
            running_loss = 0.0

    avg_loss = sum(losses)/len(losses)
    scheduler.step(avg_loss)
            
print('Training Done')

Loss [1, 100](epoch, minibatch):  8.839240016937255
Loss [1, 200](epoch, minibatch):  2.8543614387512206
Loss [1, 300](epoch, minibatch):  2.5090506088733675
Loss [2, 100](epoch, minibatch):  2.1302514731884004
Loss [2, 200](epoch, minibatch):  1.9953344476222992
Loss [2, 300](epoch, minibatch):  1.9380652523040771
Loss [3, 100](epoch, minibatch):  1.855226961374283
Loss [3, 200](epoch, minibatch):  1.7927296078205108
Loss [3, 300](epoch, minibatch):  1.743425933122635
Loss [4, 100](epoch, minibatch):  1.7187504696846008
Loss [4, 200](epoch, minibatch):  1.687288955450058
Loss [4, 300](epoch, minibatch):  1.665401849746704
Loss [5, 100](epoch, minibatch):  1.6413379180431367
Loss [5, 200](epoch, minibatch):  1.6188793969154358
Loss [5, 300](epoch, minibatch):  1.591441843509674
Loss [6, 100](epoch, minibatch):  1.582551372051239
Loss [6, 200](epoch, minibatch):  1.5571859669685364
Loss [6, 300](epoch, minibatch):  1.5410805189609527
Loss [7, 100](epoch, minibatch):  1.5283722984790802


In [39]:
correct = 0
total = 0

with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to('cuda'), labels.to('cuda')
        outputs = net(images)
        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('Accuracy on 10,000 test images: ', 100*(correct/total), '%')

Accuracy on 10,000 test images:  85.66 %
